In [59]:
import numpy as np
import pandas as pd
import datetime

In [60]:
link_df = pd.read_csv('../../data/original/links (table 3).csv', index_col=0)

In [85]:
# validation_path = '../../data/offline/validation_9_6/'
# window_date = pd.to_datetime('2016-09-06')
# prediction_file_name = 'prediction_9_6'

# validation_path = '../../data/offline/validation_9_20/'
# window_date = pd.to_datetime('2016-09-20')
# prediction_file_name = 'prediction_9_20'

validation_path = '../../data/offline/validation_10_11/'
window_date = pd.to_datetime('2016-10-11')
prediction_file_name = 'prediction_10_11'

In [86]:
afternoon_predict_dict = {}
morning_predict_dict = {}

afternoon_real_dict = {}
morning_real_dict = {}

for link_id in link_df.index.values:
    predict_afternoon_m1_df = pd.read_csv(validation_path + 'prediction/feature1/afternoon/' + str(link_id) + '.csv', index_col=0)
    
    predict_morning_m1_df = pd.read_csv(validation_path + 'prediction/feature1/morning/' + str(link_id) + '.csv', index_col=0)
    
    real_afternoon_m1_df = pd.read_csv(validation_path + 'feature/feature1/afternoon/y_test_link' + str(link_id) + '.csv', index_col = 0)
    
    real_morning_m1_df = pd.read_csv(validation_path + 'feature/feature1/morning/y_test_link' + str(link_id) + '.csv', index_col = 0)
    
    afternoon_predict_dict[link_id] = predict_afternoon_m1_df
    morning_predict_dict[link_id] = predict_morning_m1_df
    
    afternoon_real_dict[link_id] = real_afternoon_m1_df
    morning_real_dict[link_id] = real_morning_m1_df

In [87]:
def ArrayMape(ground_truth, predictions):
    diff = (ground_truth - predictions) / ground_truth
    diff = np.abs(diff)
    mape = diff.sum() / ground_truth.size
    
    return mape

In [88]:
def ScalarMape(ground_truth, predictions):
    diff = (ground_truth - predictions) / ground_truth
    diff = np.abs(diff)
    mape = diff
    
    return mape

In [92]:
routes = pd.read_csv('../../data/original/routes (table 4).csv')
submissions = []
for i in routes.index:
    
    link_seq = routes.loc[i]['link_seq']
    links = link_seq.split(',')
    
    #morning
    y_test_time = pd.read_csv(validation_path + 'feature/feature3/morning/y_test_route' + str(i) + '.csv', index_col = 0)
    y_predict_time = pd.read_csv(validation_path + 'prediction/feature3/morning/predict_' + str(i) + '.csv', index_col = 0)
    start_time = window_date + datetime.timedelta(hours=8)
    for d in range(7):
        current_time = start_time
        for t in range(6):
            predict_time = 0
            real_time = 0
            record = []
            record.append(routes.loc[i]['intersection_id'])
            record.append(routes.loc[i]['tollgate_id'])
            record.append('[' + str(current_time) + ',' + str(current_time + datetime.timedelta(minutes=20)) + ')')
            for link_id in links:
                predict_time = predict_time + morning_predict_dict[int(link_id)].loc[d][t]
                real_time = real_time + morning_real_dict[int(link_id)].loc[d][t]
                
            predict_time = predict_time + y_predict_time.loc[d][t]  / 2
            real_time = real_time + y_test_time.loc[d][t]
            
            record.append(predict_time)
            record.append(real_time)

            submissions.append(record)
            current_time = current_time + datetime.timedelta(minutes=20)
            
        start_time = start_time + datetime.timedelta(days=1)
        
    #afternoon
    y_test_time = pd.read_csv(validation_path + 'feature/feature3/afternoon/y_test_route' + str(i) + '.csv', index_col = 0)
    y_predict_time = pd.read_csv(validation_path + 'prediction/feature3/afternoon/predict_' + str(i) + '.csv', index_col = 0)
    start_time = window_date + datetime.timedelta(hours=17)
#     start_time = pd.to_datetime('2016-09-20 17:00:00')
#     start_time = pd.to_datetime('2016-10-11 17:00:00')
    for d in range(7):
        current_time = start_time
        for t in range(6):
            predict_time = 0
            real_time = 0
            record = []
            record.append(routes.loc[i]['intersection_id'])
            record.append(routes.loc[i]['tollgate_id'])
            record.append('[' + str(current_time) + ',' + str(current_time + datetime.timedelta(minutes=20)) + ')')
            for link_id in links:
                predict_time = predict_time + afternoon_predict_dict[int(link_id)].loc[d][t] 
                real_time = real_time + afternoon_real_dict[int(link_id)].loc[d][t]
            
            predict_time = predict_time + y_predict_time.loc[d][t] / 2
            real_time = real_time + y_test_time.loc[d][t] 
            
            record.append(predict_time)
            record.append(real_time)

            submissions.append(record)
            current_time = current_time + datetime.timedelta(minutes=20)
            
        start_time = start_time + datetime.timedelta(days=1)

In [93]:
submission_df = pd.DataFrame(submissions).rename(columns={0:'intersection_id', 1:'tollgate_id', 2:'time_window', 3:'predict_time', 4:'real_time'})

In [94]:
mape_sum = 0
mapes = []
for i in range(len(submission_df)):
    mape = ScalarMape(submission_df.loc[i]['real_time'], submission_df.loc[i]['predict_time'])
    #print('%a-%d : %f'%(m[0], m[1], mape))
    mapes.append(mape)
    mape_sum = mape_sum + mape
    
submission_df['mape'] = mapes
print("mape=%f"%(mape_sum / len(submissions)))

mape=0.153892


In [84]:
submission_df.to_csv('../../data/offline/prediction/m1_m3/' + prediction_file_name + '.csv')

In [ ]:
1, mape=0.242898 0.205881 0.037017
2, mape=0.190741 0.151198 0.039543
3, mape=0.185790 0.149934 0.035856

In [ ]:
9-6 mape=0.224219 0.193308
9-20 mape=0.178480 0.151585
10-11 mape=0.179806 0.153892

In [18]:
submission_df[submission_df['mape'] > 0.25].sort_values(['intersection_id', 'tollgate_id'])

,intersection_id,tollgate_id,time_window,predict_time,real_time,mape
16,A,2,"[2016-10-13 09:20:00,2016-10-13 09:40:00)",67.662847,104.038814,0.349638
19,A,2,"[2016-10-14 08:20:00,2016-10-14 08:40:00)",72.537048,109.115141,0.335225
24,A,2,"[2016-10-15 08:00:00,2016-10-15 08:20:00)",67.670174,108.723522,0.377594
26,A,2,"[2016-10-15 08:40:00,2016-10-15 09:00:00)",74.619884,142.464061,0.476220
27,A,2,"[2016-10-15 09:00:00,2016-10-15 09:20:00)",67.865027,139.360072,0.513024
28,A,2,"[2016-10-15 09:20:00,2016-10-15 09:40:00)",67.452612,149.606823,0.549134
29,A,2,"[2016-10-15 09:40:00,2016-10-15 10:00:00)",66.261673,159.099324,0.583520
36,A,2,"[2016-10-17 08:00:00,2016-10-17 08:20:00)",66.858488,130.193056,0.486467
37,A,2,"[2016-10-17 08:20:00,2016-10-17 08:40:00)",71.945652,108.881681,0.339231
46,A,2,"[2016-10-11 18:20:00,2016-10-11 18:40:00)",61.121897,155.819733,0.607740
